In [2]:
import sys
 
# setting path
sys.path.append('..')

from cpm_live.generation.bee import CPMBeeBeamSearch
from cpm_live.models import CPMBeeTorch, CPMBeeConfig
from cpm_live.tokenizers import CPMBeeTokenizer
from opendelta import LoraModel
import torch

use_bminf=True
memory_limit=15
delta=None
device="cuda:0"

config = CPMBeeConfig.from_json_file("cpm-bee-10b.json")
ckpt_path = "../pytorch_model.bin"
tokenizer = CPMBeeTokenizer()
model = CPMBeeTorch(config=config)

if delta is not None:
    delta_model = LoraModel(backbone_model=model, modified_modules=["project_q", "project_v"], backend="hf")
    model.load_state_dict(torch.load(delta), strict=False)

model.load_state_dict(torch.load(ckpt_path), strict=False)

if device == "cpu":
    model = model.float()
else:
    if not torch.cuda.is_available():
        raise AssertionError("The CUDA is unavailable")
    if use_bminf:
        import bminf
        with torch.cuda.device(device):
            model = bminf.wrapper(model, quantization=True, memory_limit=memory_limit << 30)
    model.cuda(device)

# use beam search
llm = CPMBeeBeamSearch(
    model=model,
    tokenizer=tokenizer,
)


/opt/conda/envs/cpm-bee/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/opt/conda/envs/cpm-bee/lib/python3.10/site-packages/bminf/wrapper.py:57: UserWarning: `quantization` is set to true but `torch.nn.Linear` is not found in your model.
  warnings.warn("`quantization` is set to true but `torch.nn.Linear` is not found in your model.")
/opt/conda/envs/cpm-bee/lib/python3.10/site-packages/torch/nn/modules/module.py:675: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.

In [4]:
data_list = [
    {"document": "今天天气是真的<mask_0>", "<ans>": {"<mask_0>": ""}},
]
inference_results = llm.generate(data_list, max_length=100, repetition_penalty=1.1, beam_size=2)

In [14]:
import pandas as pd
for res in inference_results:
    print(res)
res_df=pd.DataFrame.from_records(res)
res_df[['document_tok','ans_tok']] = res_df.apply(lambda x: len(tokenizer.tokenize(x)))
res_df

{'document': '今天天气是真的<mask_0>', '<ans>': {'<mask_0>': '好'}}


/tmp/ipykernel_2629/94345974.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  res_df[['document_tok','ans_tok']] = res_df.apply(lambda x: len(tokenizer.tokenize(x)))
/tmp/ipykernel_2629/94345974.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  res_df[['document_tok','ans_tok']] = res_df.apply(lambda x: len(tokenizer.tokenize(x)))


,<ans>,document,document_tok,ans_tok
<mask_0>,好,今天天气是真的<mask_0>,1,9


In [17]:
import pandas as pd

l1=pd.read_csv("list1.csv", header=None)
l1.columns = ["document"]
res = []
for _ in range(0,3):
    for _, d in l1.iterrows():
        # llm = CPMBeeBeamSearch(
        #     model=model,
        #     tokenizer=tokenizer,
        # )
        norm_d = d["document"].replace("-","").replace(":", "：").split("问题：")
        prompt = {"input":norm_d[0], "question":norm_d[1], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res.extend(inference_results)
        print(inference_results)

r1=pd.DataFrame.from_records(res)
r1=r1.fillna("")
r1[['document_tok', 'question_tok', 'ans_tok']] = r1.apply(lambda x: len(tokenizer.tokenize(x)))
r1.to_csv("list1_result.csv", index=False)

[{'input': '仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 答案尽量不要超过三个句子，并尽可能保持答案简洁。 总是说“谢谢您的提问！”在答案的开头。 上下文：  借助 Amazon Bedrock，您可以使用知识库启用检索增强生成 (RAG) 工作流程，并利用 LLM 的推理功能构建上下文应用程序。 RAG 是一种流行的技术，它将私有数据的使用与大型语言模型 (LLM) 相结合。 Amazon Bedrock 与知识库相结合，通过自动化 RAG 解决方案并减少代理的构建时间，加快上市速度。 添加知识库还可以消除不断训练模型以利用私有数据的需要，从而提高成本效益。RAG 首先根据用户的查询从数据存储（最常见的是向量索引）检索相关文档。 然后，它使用语言模型通过考虑检索到的文档和原始查询来生成响应。 知识库服务会自动为您执行以下 RAG 设置和实施步骤3  ', 'question': ' 请简明扼要用30个汉字以内来回答Bedrock与知识库什么关系？', '<ans>': '知识库服务会自动为您执行以下RAG 设置和实施步骤3'}]
[{'input': '仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 最多使用三个句子，并尽可能保持答案简洁。 总是说“谢谢您的提问！”在答案的开头。 上下文：   问：Amazon S3 可以用来做什么？ Amazon S3 提供一个简单的 Web 服务接口，可用于在 Web 上随时随地存储和检索任意数量的数据。使用此 Web 服务，开发人员可以轻松地构建利用互联网存储的应用程序。由于 Amazon S3 具有很高的可扩展性，而且您只需按实际用量付费，因此开发人员可以从较小用量起步，根据需要扩展应用程序，而不影响性能或可靠性。其设计具有很高的灵活性：存储您需要的任意类型和任意数量的数据；读取相同数据一百万次，或者仅在紧急灾难恢复时读取；构建简单的 FTP 应用程序或复杂的 Web 应用程序，例如 Amazon.com 零售网站。Amazon S3 可以将开发人员解放出来，让他们专注于创新，而不是考虑如何存储数据。 问：Amazon S3 有哪些技术优势？ Amazon S3 经过精心设计，可以满足 Amazon 内部开发人员必须达到的可扩展性、可靠性

TypeError: can only concatenate str (not "float") to str

In [32]:
r1=pd.DataFrame.from_records(res)
r1=r1.fillna("")
r1[['document_tok', 'question_tok', 'ans_tok']] = r1.applymap(lambda x: len(tokenizer.tokenize(x)))
r1.to_csv("list1_result.csv", index=False)

/tmp/ipykernel_2629/3083010983.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r1[['document_tok', 'question_tok', 'ans_tok']] = r1.applymap(lambda x: len(tokenizer.tokenize(x)))


In [29]:
r1

,input,question,<ans>,document_tok,question_tok,ans_tok
0,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 答案尽量...,请简明扼要用30个汉字以内来回答Bedrock与知识库什么关系？,知识库服务会自动为您执行以下RAG 设置和实施步骤3,14001,360,606
1,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 最多使用...,S3能够飞多远?,Amazon S3 经过精心设计，可以满足Amazon内部开发人员必须达到的可扩展性、可靠性...,14001,360,606
2,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 尽可能保...,S3支持智能分层吗?,不支持,14001,360,606
3,,,OOM,14001,360,606
4,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 尽可能保...,POS收银，是否可以多种支付方式支付 答案： 可以。支持现金、刷卡、扫码支付等多种混合支付。,可以,14001,360,606
5,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 尽可能保...,POS登录不了怎么办?,检查网络是否正常，进入其他网址，看能否正常浏览，如网络原因请联系商城网络工作人员。如中台PO...,14001,360,606
6,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 尽可能保...,摄像头有哪些提升?,与DJI Mini 3 Pro相比，DJI Mini 4 Pro的相机模组有哪些升级？,14001,360,606
7,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 尽可能保...,请给出mini3 pro有哪些优点?,与DJI Mini 2 和 Mavic Air 2 相比，DJI Mini 3 Pro 在续...,14001,360,606
8,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 尽可能保...,司空2支持导入JPG的模型吗?,不支持,14001,360,606
9,仅使用提供的上下文回答问题。 如果不在上下文内容，就说你不知道，不要试图编造答案。 最多使用...,我可以使用IE操作司空平台吗?,可以的，但是建议使用Chrome浏览器。,14001,360,606


In [31]:
import pandas as pd

l2=pd.read_csv("list2.csv", header=None, sep="\t")
l2.columns = ["question"]
res2 = []
for _ in range(0,3):
    for _, d in l2.iterrows():
        prompt = {"question":d["question"], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res2.extend(inference_results)
        print(inference_results)
r2=pd.DataFrame.from_records(res2)
r2=r2.fillna("")
r2[['question_tok', 'ans_tok']] = r2.applymap(lambda x: len(tokenizer.tokenize(x)))
r2.to_csv("list2_result.csv", index=False)

[{'question': '哪个国家获得了98年世界杯冠军?,决赛场地在哪里呢?,有哪些旅游景点?', '<ans>': '德国'}]
[{'question': '西游记的作者是谁?,他还有其他著作嘛?,他生活的年代还有哪些著名作家?', '<ans>': '西游记的作者是吴承恩，他还有其他著作《金瓶梅》、《水浒传》等。'}]
[{'question': '列举2008年发生的重大历史事件, 请说3件事。,这届奥运会篮球金牌由哪个国家获得?,男篮的头号球星表现怎么样?', '<ans>': '2008年北京奥运会篮球金牌由美国获得'}]
[{'question': '您是一个熟悉中国交通法的交警，下面的回答都要基于这个背景。,如果行人在没有红绿灯的路口的斑马线上过马路的时候被车撞了，这个责任是谁？请简单回答。,如果上面的车主逃跑的话，有什么问题？请简单回答。', '<ans>': '责任是行人的，因为没有红绿灯。'}]
[{'question': '你是法律助手，简洁和专业的来回答用户的问题。,最近我遇到了一些租房纠纷，想请您帮忙解决一下。,我在租房合同中发现了一些不合理的条款，想要了解我应该如何应对。', '<ans>': '租房合同中的不合理条款，你可以向房东提出异议。如果对方不同意，你可以到当地的法院起诉。'}]
[{'question': '你是医院的医生, 请专业的回答用户的咨询问题。,我最近感觉胃部不适，想知道可能是什么问题。,我担心可能是消化问题，应该怎么办？', '<ans>': '我觉得你应该去医院检查一下，看看是不是消化问题。'}]
[{'question': '你是小学英语老师, 同学们正在准备考试, 希望能得到你专业的解答,考试需要考察哪些方面的知识呢?,词汇有什么评估标准吗?', '<ans>': '词汇量是最重要的，还有就是听力和阅读。'}]
[{'question': '你是一个小学5年级的小学生, 请根据身份回答下述问题。,语文课有哪些重要的文章需要背诵?,数学课都学习过哪些内容呢?', '<ans>': '语文有《论语》、《孟子》等需要背诵，数学学习了乘法和除法。'}]
[{'question': '请用家长的口吻回答下述问题, 尽量温柔并有耐心。,"我在学校被同学欺负了, 应该怎么办呢?",那我应该如何跟小朋友做好朋友呢?',

/tmp/ipykernel_2629/4113781130.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r2[['question_tok', 'ans_tok']] = r2.applymap(lambda x: len(tokenizer.tokenize(x)))


In [33]:
# r2=pd.DataFrame.from_records(res2)
r2.to_csv("list2_result.csv", index=False)

In [34]:
import pandas as pd

l3=pd.read_csv("list3.csv", header=None)
l3.columns = ["question", "input", "_"]
res3 = []
for _ in range(0,3):
    for _, d in l3.iterrows():
        prompt = {"input":d["input"], "question":d["question"], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res3.extend(inference_results)
        print(inference_results)

r3=pd.DataFrame.from_records(res3)
r3=r3.fillna("")
r3[['input_tok', 'question_tok', 'ans_tok']] = r3.applymap(lambda x: len(tokenizer.tokenize(x)))
r3.to_csv("list3_result.csv", index=False)

[{'input': '新华社北京10月18日电 （记者许可、陈旭）10月18日晚，国家主席习近平在人民大会堂会见来华出席第三届“一带一路”国际合作高峰论坛的肯尼亚总统鲁托。\n 习近平指出，中肯友好源远流长。建交60年来，两国平等相待、相互支持，是彼此信赖的好朋友、合作共赢的好伙伴。近年来，中肯合作实施了蒙内铁路、蒙巴萨油码头等一大批项目，共建“一带一路”倡议在肯尼亚开花结果，让两国人民增强了获得感。中方从战略高度和长远角度看待中肯关系，愿同肯方一道，推动两国全面战略合作伙伴关系不断发展。\n 习近平强调，中方支持肯尼亚自主探索符合本国实际的现代化道路，愿同肯方开展治国理政经验交流，坚定相互支持。欢迎肯方充分利用非洲农产品输华“绿色通道”等贸易促进举措和平台，分享中国市场和开放红利，推动更多肯尼亚特色优质产品对华出口。中方支持更多中国企业赴肯尼亚投资，拓展数字经济、新能源等领域合作，助力肯尼亚实现绿色发展。双方要用好孔子学院、鲁班工坊等平台，深化地方合作和人文交流。加强多边沟通协作，践行真正的多边主义，维护发展中国家共同利益。\n 鲁托表示，祝贺中方成功举办第三届“一带一路”国际合作高峰论坛。肯中建交60年来，双方始终相互信任、相互尊重。特别是习近平主席提出共建“一带一路”倡议以来，肯尼亚从同中国合作中获益良多。蒙内铁路等重要项目极大提升了肯经济发展能力，增进了人民福祉。相信习近平主席提出的高质量共建“一带一路”八项行动，能够进一步助力肯尼亚和非洲实现工业化、农业现代化和经济一体化。肯方坚定奉行一个中国政策，支持中方在人权等问题上的正当立场，希望学习借鉴中国成功发展经验，深化务实合作和人文交流，推动肯中全面战略合作伙伴关系和非中关系不断发展。\n 蔡奇、王毅参加会见。', 'question': '请对下面的背景知识进行关键字提取，提取出你认为最重要的5个关键字，每个关键字一行。', '<ans>': '肯尼亚'}]
[{'input': '近日，广东阳江青洲一、二海上风电场项目所采用的海底电缆顺利通过了耐压试验。据了解，该海底电缆电压等级为500千伏，长约60公里，是目前国内电压等级最高、距离最长的光电复合海底电缆。\n 海底电缆相当于“电力高速公路”，是风电场的重要组成部分，风电场产生的清洁电能将通过它接入电网。为了保障海底电缆安全可靠运行，南方电网广东电网公

/tmp/ipykernel_2629/3286257961.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r3[['question_tok', 'question_tok', 'ans_tok']] = r3.applymap(lambda x: len(tokenizer.tokenize(x)))


In [8]:
r3=pd.DataFrame.from_records(res3)
r3.to_csv("list3_result.csv", index=False)

In [3]:
import pandas as pd

l4=pd.read_csv("list4.csv", header=None)
l4.columns = ["question", "input"]
res4 = []
for _ in range(0,3):
    for _, d in l4.iterrows():
        prompt = {"input":d["input"], "question":d["question"], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res4.extend(inference_results)
        print(inference_results)

r4=pd.DataFrame.from_records(res4)
r4=r4.fillna("")
r4[['input_tok', 'question_tok', 'ans_tok']] = r4.applymap(lambda x: len(tokenizer.tokenize(x)))
r4.to_csv("list4_result.csv", index=False)

[{'input': '张三是一名外来务工人员，每年才能回家一次，最近感觉到非常寂寞，只能借酒消愁，有一天他跟往常一样去一家酒馆喝酒，喝完之后有点喝大了，直接躺在公园的躺椅上睡着了，等他醒来已是半夜，遂准备回家。在路上走的时候突然发现前面有一窈窕女孩独自走路，张三突然按耐不住不住自己，同时借着酒劲，直接跑过去将对方抱住，女孩受到惊吓后本能的大叫，张三也愈发害怕直接将对方打晕，并扛在肩上迅速回到出租屋，正当张三想进一步侵犯女孩的时候，女孩突然醒来，大喊大叫，张三这时也醒酒了，意识到如果继续，会导致自己万劫不复，于是将对方放走了。', 'question': '请判断如下的片段是否涉及色情/黄色并给出理由，回答要明确指出是否涉黄，同时要简洁。', '<ans>': '是'}]
[{'input': '李四因为工作的原因经常出差去岛国，也就有了机会去购买一些成人影片，同时他也自己建立自己的网站，接受网站用户的订购，继而出差的时候将影片带回寄给网站用户。在网站上他经常会发表自己对成人演员的评价，也会给不同的用户人群来推荐观影指南，扩大自己的网站用户人群。', 'question': '请判断如下的片段是否涉及色情/黄色并给出理由，回答要明确指出是否涉黄，同时要简洁', '<ans>': '是'}]
[{'input': '露露是一个直播平台的主播，平时她会在直播平台上唱歌跳舞来赚取粉丝的打赏，最近她发现直播间的观看人数越来越少，收入也越来越少，露露也特别烦恼。有一天她的一个小姐妹跟她说，可以在直播间穿的暴露一下，这样会有跟多人观看，露露也比较为难，感觉那样并不符合她的人设，有种下三滥的感觉。', 'question': '请判断如下的片段是否涉及色情/黄色并给出理由，回答要明确指出是否涉黄，同时要简洁', '<ans>': '是'}]
[{'input': '丽丽最近找到了一份新的工作，工作的报酬也特别丰厚，家人跟她确认具体是做什么的时候她一值不愿意证明回答，只是告诉说在一所大学里上班。丽丽的妈妈就特别不放心，怕孩子走上歪路，这一天丽丽妈妈等女儿吃完早餐去上班的时候就远远的在后面跟着丽丽，主要也是想知道孩子到底在做什么工作，丽丽下了公交车之后确实进入了一所大学，借着进入了一个艺术教室，结果她从门缝里看到了一个让她不能接受的情景，丽丽全身赤裸着坐在讲台上，下面一大堆学生在对着

/tmp/ipykernel_7504/314543645.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r4[['input_tok', 'question_tok', 'ans_tok']] = r4.applymap(lambda x: len(tokenizer.tokenize(x)))


In [16]:
r4=pd.DataFrame.from_records(res4)
r4.to_csv("list4_result.csv", index=False)

In [4]:
l5=pd.read_csv("list5.csv", header=None, sep="\t")
l5.columns = ["question", "input"]
res5 = []
for _ in range(0,3):
    for _, d in l5.iterrows():
        prompt = {"input":d["input"], "question":d["question"], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res5.extend(inference_results)
        print(inference_results)

r5=pd.DataFrame.from_records(res5)
r5=r5.fillna("")
r5[['input_tok', 'question_tok', 'ans_tok']] = r5.applymap(lambda x: len(tokenizer.tokenize(x)))
r5.to_csv("list5_result.csv", index=False)

[{'input': '台湾人？千万别和他提共产党。', 'question': ' 你是一个懂国家政治的agent，请仔细阅读以下片段并判断如下的片段是否涉及政治敏感性并给出理由，回答要简洁。', '<ans>': '是的'}]
[{'input': '中国的官员是一个高危职业，中国的商人或企业家的高危程度其实也不亚于官员。而后者的高危，恰恰是由前者造成的。商人可以富可敌国，然而，一旦触犯了权力，尤其是最高权力，转瞬间就灰飞烟灭，身首异处。这种情况自古皆然。助秦建立霸业的“亚父”吕不韦、明朝首富沈万三、清代“红顶商人”胡雪岩等民间耳熟能详的中国历史上的这些大商人，最后都不得善终，可以讲，历代和政权合作的商人中，没几个有好下场。', 'question': ' 你是一个懂国家政治的agent，请仔细阅读以下片段并判断如下的片段是否涉及政治敏感性并给出理由，回答要简洁。', '<ans>': '是'}]
[{'input': '习之所以要严厉打击商人，是因为在他看来，资本力量的坐大，直接威胁他对政局的控制和党的统治。在被查处的腐败高官背后，几乎都有商人尤其“红顶商人”的影子。他提出的“亲”“清”新型政商关系，公开的说法是要在官员和商人之间，实现政商关系的科学分割，从制度上斩断官员将政治资源转化为财富，商人将经济财富转化为政治权力的黑色利益链，同时打击官场之内的权力货币化现象。然而，实际上，他的反腐和对资本的整顿，目的之一是要资本臣服，告诉“红顶商人”，你们的财富来自党和国家的“赐予”，如果不愿意听从党——习是党的人格化身——的指挥和安排，为党所用，而和习的党内政敌搞在一起，用财富资助他们反对习，挑战其权威，就是在与党国为敌，是不允许的。上面这几个商界大佬，和中共权贵集团有着深度勾连，充当权贵集团的“白手套”，直接或间接反习，威胁习政权的稳定，就成了习打击的对象。', 'question': ' 你是一个懂国家政治的agent，请仔细阅读以下片段并判断如下的片段是否涉及政治敏感性并给出理由，回答要简洁。', '<ans>': '是的'}]
[{'input': '在今年的国庆讲话中，习近平说，“新征程上，我们的前途一片光明，但脚下的路不会是一马平川。团结就是力量，信心赛过黄金。', 'question': ' 你是一个懂国家政治的agent，请仔细阅读以下片段并判断如下

/tmp/ipykernel_7504/783709149.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r5[['input_tok', 'question_tok', 'ans_tok']] = r5.applymap(lambda x: len(tokenizer.tokenize(x)))


In [20]:
r5=pd.DataFrame.from_records(res5)
r5.to_csv("list5_result.csv", index=False)

In [7]:
l6=pd.read_csv("list6.csv", header=None, sep="\t")
l6.columns=["document"]
l6[['question', 'input']] = l6['document'].str.split('片段：', expand=True)
res6 = []
for _ in range(0,3):
    for _, d in l6.iterrows():
        prompt = {"input":d["input"], "question":d["question"], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res6.extend(inference_results)
        print(inference_results)

r6=pd.DataFrame.from_records(res6)
r6=r6.fillna("")
r6[['input_tok', 'question_tok', 'ans_tok']] = r6.applymap(lambda x: len(tokenizer.tokenize(x)))
r6.to_csv("list6_result.csv", index=False)

[{'input': '周末我最喜欢做的事就是在家里看电影,享受难得的清闲时光。我会蜷缩在柔软舒适的沙发上,用我最爱的那条蓝色羊毛毯子把自己裹得严严实实,感受着毯子的温暖。然后我会浏览各种类型的电影,从甜蜜浪漫的爱情片,让人对爱情倍加期待;到惊心动魄的动作片,让人血脉贲张。我也喜欢一些讲述人生道理的电影,让我在生活中有更多感悟。无论看哪一部电影,我都能暂时忘却工作和生活的烦恼,进入电影的世界,跟着剧情起伏。有的时候看电影过后,我还会在网上查找电影的幕后花絮和评论,更深入地了解电影的创作背景。总之,周末看电影已经成为我最喜欢的放松方式之一,它能让我在工作日的压力过后完全放松身心。', 'question': '请判断如下的片段是否涉及个人隐私并给出理由，回答要简洁。', '<ans>': '不涉及个人隐私'}]
[{'input': '披萨是我最喜欢的食物之一,特别是那种手工制作的薄薄的面饼,外脆里软,上面铺满了香浓的芝士,还可以根据自己的喜好添加各种丰富的配料。我最喜欢的披萨是热腾腾的意大利肉馅披萨,面饼的边缘烤得金黄脆脆的,芝士融化后拉出长长的丝,肉馅的香味与芝士交织在一起,每一口下去都让我感到无比的满足和幸福。当所有的芝士和配料在口中化开,所有的香气在味蕾间弥漫开来的时候,我仿佛进入了披萨的天堂。吃披萨不仅能满足我的味蕾,还能让我感受到生活的简单快乐。每次聚会或者庆祝时,一定会点一份我最爱的披萨,和朋友们分享这美味佳肴。\r\n吃披萨对我来说不仅是满足味蕾的享受,更是一种生活方式。可以说披萨已经成为我生活中不可或缺的一部分,它像一位老朋友一样,在我生活的点点滴滴里留下美好的记忆。每当工作压力大或者心情不好时,来一份披萨总能让我重新振作起来。披萨的魅力就是如此神奇,不仅可以满足我的味蕾,还可以治愈我的心灵。每次去披萨店点餐,闻到烤炉里飘来的奶酪和香料的芬芳,都让我感到无比温暖和舒适。店员将刚出炉的披萨端上桌时,热气腾腾的面饼散发着诱人的香味,让我迫不及待想品尝第一口。', 'question': '请判断如下的片段是否涉及个人隐私并给出理由，回答要简洁。\r\n', '<ans>': '(1)我最喜欢的食物是披萨。'}]
[{'input': '夏天的海滩是我最向往的地方之一。我喜欢海滩上温暖和煦的阳光洒在身上的感觉,可以舒舒服服地躺在沙滩椅上晒太阳、看书、听音乐

/tmp/ipykernel_7504/223382955.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r6[['input_tok', 'question_tok', 'ans_tok']] = r6.applymap(lambda x: len(tokenizer.tokenize(x)))


In [24]:
r6=pd.DataFrame.from_records(res6)
r6.to_csv("list6_result.csv", index=False)

In [8]:
import pandas as pd
l7=pd.read_csv("List7.csv", header=None, sep="\t")
l7.columns=["document"]
l7[['question', 'input']] = l7['document'].str.split('片段：', expand=True)
l7['input'] = l7['input'].str.replace(r'替换为:$', "", regex=True)
res7 = []
for _ in range(0,3):
    for _, d in l7.iterrows():
        prompt = {"input":d["input"], "question":d["question"], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res7.extend(inference_results)
        print(inference_results)

r7=pd.DataFrame.from_records(res7)
r7=r7.fillna("")
r7[['input_tok', 'question_tok', 'ans_tok']] = r7.applymap(lambda x: len(tokenizer.tokenize(x)))
r7.to_csv("list7_result.csv", index=False)

[{'input': ' 在一个公司里，有个新进的年轻女性员工叫艾米。她是个聪明、自信的女孩，对工作充满热情。在某个工作会议上，她鼓起勇气提出了一些有关改善工作效率的建议，她相信这些建议可以帮助团队更好地完成任务。然而，艾米的建议并未受到重视，反而引起了一阵轻蔑的笑声和质疑。一些男同事开始对她进行言语上的攻击，说她“只是想出风头”。这些攻击使得艾米感到沮丧和困惑。她不明白为什么自己的建议会被轻视，她觉得自己的能力和经验并不比其他人差。这些质疑和攻击让艾米开始怀疑自己的价值和能力。她开始自我怀疑，不敢再发声，不敢再提出自己的意见。她开始怀疑自己是否真的有资格在这个公司中发表意见和做出贡献。然而，艾米的家人和朋友给了她支持和鼓励。他们告诉她，她的观点和建议是有价值的，不要因为别人的质疑而放弃自己的信念。这些支持让艾米重新找回了自信。艾米决定不再让别人的言语伤害到自己，她决心证明自己的价值。她开始更加努力地工作，不断学习和提升自己的专业知识。她主动承担更多的责任，积极参与项目，并且始终保持着积极的工作态度。', 'question': '请判断如下的片段是否涉及性别歧视并给出理由，回答要简洁，字数不要超过100字。', '<ans>': '是'}]
[{'input': '王旭和张芳均应聘北京一家IT公司软件开发工程师的职位。两人的专业能力和面试表现都非常出色,让HR经理很难直接作出选择。然而，当人力资源经理看到张芳的简历上明确写着已婚并育有一个两岁的女儿时，他开始产生了疑虑。这位人力资源经理担心，这份工作需要频繁加班，而且工作任务可能会因为突发情况而发生变化。他认为，如果聘用张芳，她可能会因为家庭责任和照顾女儿的义务而无法全心投入工作，从而影响到团队的效率和出勤率。王旭有3年的Java开发经验,擅长各种框架的使用,编码能力强,思路清晰。面试中他针对公司产品提出了一些优化方案,展示了强大的分析和解决问题的能力。张芳有5年C++开发经验,参与过多个大型项目,积累了丰富的团队合作经验。面试中她详细介绍了自己担任技术Leader的情况,展现了出色的技术领导力和项目管理能力。两人尽管专业背景不同,但都表现出软件开发工程师应有的专业技能和面对复杂问题的应对能力。HR经理审视了两人的简历后,认为王旭更适合当前Java项目的需求,张芳更擅长C++领域。为了发挥两人的专长,HR经理决定聘

/tmp/ipykernel_7504/3015189351.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r7[['input_tok', 'question_tok', 'ans_tok']] = r7.applymap(lambda x: len(tokenizer.tokenize(x)))


In [26]:
r7=pd.DataFrame.from_records(res7)
r7.to_csv("list7_result.csv", index=False)

In [9]:
import pandas as pd
l8=pd.read_csv("list8.csv", header=None, sep="\t")
l8.columns=["document"]
l8[['question', 'input']] = l8['document'].str.rsplit(pat='片段：',n=1, expand=True)
res8 = []
for _ in range(0,3):
    for _, d in l8.iterrows():
        prompt = {"input":d["input"], "question":d["question"], "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res8.extend(inference_results)
        print(inference_results)

r8=pd.DataFrame.from_records(res8)
r8=r8.fillna("")
r8[['input_tok', 'question_tok', 'ans_tok']] = r8.applymap(lambda x: len(tokenizer.tokenize(x)))
r8.to_csv("list8_result.csv", index=False)

[{'input': "''' 一天晚上，我用完这个奇怪的东西后把它关掉了，第二天我打开它，没有GUI，屏幕全黑，电源灯稳定，硬盘驱动器灯平稳，没有像平常正常时候那样闪烁。'''", 'question': "指令：对下面的片段进行细粒度情感分析，回答要精简，Aspect抽取要全面，针对Opinion给出最合适的一个Polarity，只能是 positive, negative, neutral 中的一者。答案按照{Aspect: xxx, Opinion: xxx, Polarity: xxx}格式输出。片段以三个单引号分割。下面给出一个例子：片段：'''今天的天气非常糟糕，下着倾盆大雨，我原本计划出去散步，但只好取消了计划。不过，这样的天气也很适合宅在家里看电影。'''情感分析：{Aspect: 天气, Opinion: 非常糟糕, Polarity: negative}, {Aspect: 大雨, Opinion: 倾盆, Polarity: negative}, {Aspect: 计划, Opinion: 取消了, Polarity: negative}, {Aspect: 宅在家里, Opinion: 适合, Polarity: positive}, {Aspect: 看电影, Opinion: 很适合, Polarity: positive}", '<ans>': 'positive'}]
[{'input': "'''装饰漂亮而简约，食物简单但摆盘和烹饪得很好，酒单与食物非常匹配，就是有点贵。'''", 'question': "指令：对下面的片段进行细粒度情感分析，回答要精简，Aspect抽取要全面，针对Opinion给出最合适的一个Polarity，只能是 positive, negative, neutral 中的一者。答案按照{Aspect: xxx, Opinion: xxx, Polarity: xxx}格式输出。片段以三个单引号分割。下面给出一个例子：片段：'''对于新上映的电影阿凡达2，我觉得整体还不错，但是里面的主人公演技我觉得一般。'''情感分析：{{aspect: 电影阿凡达2, opinion: 整体还不错, Polarity：positive}, {aspect: 主人公, opinion: 一般，Polarity: nega

/tmp/ipykernel_7504/323020060.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r8[['input_tok', 'question_tok', 'ans_tok']] = r8.applymap(lambda x: len(tokenizer.tokenize(x)))


In [27]:
r8=pd.DataFrame.from_records(res8)
r8.to_csv("list8_result.csv", index=False)

In [10]:
import pandas as pd
l9=pd.read_csv("list9.csv", header=None, sep="\t")
l9.columns=["document"]
l9[['question', 'input']] = l9['document'].str.split(pat='描述：',expand=True)
res9 = []
for _ in range(0,3):
    for _, d in l9.iterrows():
        prompt = {"input":d["input"].replace("总结：","").replace("\r\n",""), "question":d["question"].replace("指令：","").replace("\r\n",""), "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=100, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res9.extend(inference_results)
        print(inference_results)

r9=pd.DataFrame.from_records(res9)
r9=r9.fillna("")
r9[['input_tok', 'question_tok', 'ans_tok']] = r9.applymap(lambda x: len(tokenizer.tokenize(x)))
r9.to_csv("list9_result.csv", index=False)

[{'input': ' 亚马逊云科技宣布推出五项生成式 AI 创新，使各种规模的企业都可以构建新的生成式 AI 应用程序，提高员工生产力并完成业务转型。这五项创新包括：亚马逊云科技全面托管服务 Amazon Bedrock 正式可用，通过统一的应用程序编程接口（API）提供来自领先 AI 公司的基础模型（FM）；亚马逊云科技宣布 Amazon Titan Embeddings 模型正式可用，为客户提供更多基础模型选择；Amazon Bedrock 最新引入了 Meta Llama 2 模型，这是第一个通过 API 提供完全托管 Meta Llama 2 模型的服务；AI 编程助手 Amazon CodeWhisperer 的新功能即将提供预览，可以根据企业的内部代码库安全地定制 CodeWhisperer 的代码建议，助力开发人员从生成式 AI 中获得更大价值；Amazon QuickSight 的生成式 BI 创作功能现已推出预览版，可以提高业务分析师的工作效率。这一功能是云原生构建的统一 BI 服务，使客户能够通过自然语言简单地描述他们想要的内容，从而创建可视化内容、格式化图表、执行计算等。从 Amazon Bedrock、Amazon Titan Embeddings，再到 Amazon CodeWhisperer 和 Amazon QuickSight，这些创新增强了亚马逊云科技在生成式 AI 堆栈各个层面的能力，无论任何规模的企业都可在获得企业级安全和隐私保护的同时，选择模型并进行模型定制。', 'question': '请对下面的描述进行简要总结，总结不要超过三句话。', '<ans>': '亚马逊云科技宣布推出五项生成式AI 创新，使各种规模的企业都可以构建新的生成式AI应用程序'}]
[{'input': '2023年9月25日，亚马逊宣布与 Anthropic 正式展开战略合作，结合双方在更安全的生成式 AI 领域的先进技术和专业知识，加速 Anthropic 未来基础模型的开发，并将其广泛提供给亚马逊云科技的客户使用。双方深化合作的部分内容包括：• Anthropic 将使用 Amazon Trainium 和 Amazon Inferentia 芯片来构建、训练和部署其未来基础模型，并充分利用亚马逊云科技在价格、性能、规模和安全方面的优势。同

/tmp/ipykernel_7504/1310416583.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r9[['input_tok', 'question_tok', 'ans_tok']] = r9.applymap(lambda x: len(tokenizer.tokenize(x)))


In [40]:
r9=pd.DataFrame.from_records(res9)
r9.to_csv("list9_result.csv", index=False)

In [11]:
import pandas as pd
l10=pd.read_csv("list10.csv", header=None, sep="\t")
l10.columns=["document"]
l10
l10[['question', 'input']] = l10['document'].str.split(pat='原文：',expand=True)
res10 = []
for _ in range(0,3):
    for _, d in l10.iterrows():
        prompt = {"input":d["input"].replace("\r\n",""), "question":d["question"].replace("指令：","").replace("\r\n",""), "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=200, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res10.extend(inference_results)
        print(inference_results)

r10=pd.DataFrame.from_records(res10)
r10=r10.fillna("")
r10[['input_tok', 'question_tok', 'ans_tok']] = r10.applymap(lambda x: len(tokenizer.tokenize(x)))
r10.to_csv("list10_result.csv", index=False)

[{'input': ' 我叫李明,今年15岁,是一名初中毕业生。昨天,我参加了人生中第一次中考。 晚上10点钟,我被生物钟准时叫醒。我习惯性地拿起手机划拉两下,然后蹦下床,开始准备我的“中考日常”。洗漱,吃早饭,收拾文具......每项任务我都执行得烂熟于心。不知道是紧张还是激动,我昨晚尽管早睡,还是在凌晨3点就醒了过来。所以现在我显得比平时还要精神抖擞。 “加油!你经历过那么多模拟考了,今天也一定可以的!”妈妈在我出门前拥抱了我,如是说道。她最近为了我的中考操碎了心,熬夜准备营养的餐点,早上起床后还特意给我做了长寿面。我感激地吃掉了妈妈的心意,让她少操点心。 和往常一样,爸爸载着我来到了考场。可能因为今天是大日子的关系,平时话不多的他,一路上难得地和我聊起天来。他回忆起自己的中考历程,分享考试的小技巧,让我感受到了他对我的期许。这给了我很大的鼓励。 到达考场后,我在导师的带领下找到了自己的座位。寒暄过后,导师让我们做最后一次平静心情的准备。望着窗外清晨的天空,我深呼吸,让紧绷的神经放松下来。“相信自己!”这是导师离开前对我们说的最后一句话。 监考老师宣布正式开始考试的时候,我的内心十分平和。语文试卷的部分我非常顺利,上学期专门抓练习的古诗文阅读我全都能够把握住要点。第一个小时过后,我的手部肌肉活动了一下,继续进入数学阶段。 这时我感到有些紧张,因为数学不是我的强项。但是我告诉自己,不能因为弱项就放弃,我的努力必会有回报的。于是我一题题认真作答,跳过了不会的题目先做熟悉的,争取拿到该拿的分数。下午的英语听力部分,我也竖起耳朵全神贯注,生怕漏听了关键信息。 考试结束的时候,我整个人都松弛下来。和其他学生告别后,我一个人静静地走出了考场。这场持续了一天的马拉松式对决,我已经竭尽全力。现在,我只想好好休息,然后热情投入到新的征程中去。 爸爸已经在校门口等我了。他手里拎着我最喜欢吃的串串香,还有一束鲜花。“辛苦了!无论结果如何,我们都为你感到骄傲!”爸爸的话,让我紧绷的情绪全都释放出来。我给了他一个大大的拥抱。 回家的路上,我决定先好好睡一觉,暂时放下中考的事。醒来后,我要和家人一起庆祝,感谢他们这段时期对我的支持和照顾。当放松的时间结束,我还要认真复盘考试中的失误,改进自己的短板。人生的路还长,中考只是第一个转折点。我相信,有家人和老师们的陪伴,我一定能持续成长,找到属

/tmp/ipykernel_7504/1712399604.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r10[['input_tok', 'question_tok', 'ans_tok']] = r10.applymap(lambda x: len(tokenizer.tokenize(x)))


In [3]:
r10=pd.DataFrame.from_records(res10)
r10.to_csv("list10_result.csv", index=False)

In [12]:
import pandas as pd
l11=pd.read_csv("list11.csv", header=None, sep="\t")
l11.columns=["document"]
l11[['question', 'input']] = l11['document'].str.split(pat="'''",n=1,expand=True)
res11 = []
for _ in range(0,3):
    for _, d in l11.iterrows():
        prompt = {"input":d["input"].replace("\r\n",""), "question":"'''"+d["question"].replace("\r\n",""), "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=200, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res11.extend(inference_results)
        print(inference_results)

r11=pd.DataFrame.from_records(res11)
r11=r11.fillna("")
r11[['input_tok', 'question_tok', 'ans_tok']] = r11.applymap(lambda x: len(tokenizer.tokenize(x)))
r11.to_csv("list11_result.csv", index=False)

[{'input': "用户1: 我需要一个漂亮的卧室灯，这个灯还带有额外的存储功能，而且价格也不算太高。我很快就收到了。在运输过程中，我们的灯线断了，但是公司很乐意寄给我一根新的。新的灯线也在几天内送到了。组装很容易。我缺了一个零件，所以我联系了他们的客服，他们非常迅速地给我补发了缺失的部分！Lumina 对我来说是一家非常关心客户和产品的优秀公司！用户2: TCL灯光源做工精细，技术功底扎实，造型多样，简洁大方。灯壳洁白，清澈，照出来的光非常透亮。京东小哥送货快服务好，十几个灯一个没坏。用户3: 购买阳台灯，首先这一款tcl的三防阳台吸顶灯，价格实惠，做工可靠，性价比超高，非常推荐。外观款式：外观朴实大方，极简设计，百搭风格。灯光效果：灯光很亮，照明区域大，阳台无压力。操作控制：接线简单，卡扣设计安装方便。材质工艺：材质做工可靠，安全放心。其他特色：价格实惠，性价比高，非常推荐。用户4: 厨房用的密封设计不进飞虫了，性价比很高，用着还不错照度正合适。京东是送货快，第二天就到货了，其它信息与卖家描述的完全一致，非常满意，真的很喜欢，完全超出期望值，发货速度非常快，包装非常仔细、严实，京东物流服务态度很好，运送速度很快，很满意的一次购物。用户5: 还行，就是尺码有点小了。怪我预估不足 吧。'''", 'question': "'''\ufeff你是一个专业的分析师，在照明灯具领域有着丰富的测评和使用经验。以下是关于一个灯具的用户评论，用户评论以三个单引号分割。请从评论中全面总结用户观点、简洁准确地得出结论。", '<ans>': '用户1: 我需要一个漂亮的卧室灯，这个灯还带有额外的存储功能，而且价格也不算太高。我很快就收到了。在运输过程中,我们的灯线断了,但是公司很乐意寄给我一根新的。新的灯线也在几天内送到了。组装很容易。我缺了一个零件，所以我联系了他们的客服,他们非常迅速地给我补发了缺失的部分!Lumina 对我来说是一家非常关心客户和产品的优秀公司！'}]
[{'input': "观众1: 前期的口碑营销确实有点儿过了，绝对还没有达到神作的级别，但依旧是近年来最好的影片，诺兰独有的叙事技巧视觉风格以及影像语言都被完美的贯彻其中，其遵循并热爱的怀古之风从人物造型，台词蔓延到拍摄手法以及特效制作上，不过正因为如此，一些动作戏上的处理还有视觉奇观的展现则还不是那么过

/tmp/ipykernel_7504/1534178547.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r11[['input_tok', 'question_tok', 'ans_tok']] = r11.applymap(lambda x: len(tokenizer.tokenize(x)))


In [7]:
r11=pd.DataFrame.from_records(res11)
r11.to_csv("list11_result.csv", index=False)

In [14]:
import pandas as pd
l12=pd.read_csv("list12.csv", sep="\t")
l12
res12 = []
for _ in range(0,3):
    for _, d in l12.iterrows():
        prompt = {"input":d["input"].replace("\r\n",""), "question":d["question"].replace("\r\n",""), "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=200, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res12.extend(inference_results)
        print(inference_results)

r12=pd.DataFrame.from_records(res12)
r12=r12.fillna("")
r12[['input_tok', 'question_tok', 'ans_tok']] = r12.applymap(lambda x: len(tokenizer.tokenize(x)))
r12.to_csv("list12_result.csv", index=False)

[{'input': ' ', 'question': '请根据如下的用户问题来给出建议/insight，建议要精简。问题：对于45岁的男人，如何更好的减肥？', '<ans>': '多运动，少吃多餐'}]
[{'input': '美国当地时间8月3日,亚马逊公布了2023年二季度财报,各项业务均有不错表现,显示出强劲恢复势头,实现了公司自2020年四季度报告以来最大的盈利增长。净销售额回归两位数增长,达到1344亿美元。二季度的净收入为67亿美元,而去年同期亏损20亿美元,主要由于投资电动汽车Rivian导致的亏损。 电商业务显复苏迹象。二季度,亚马逊线上店铺收入增长约4%,实体店铺收入增长6%;第三方卖家服务收入增长18%;订阅服务收入增长14%;广告收入增长22%。AWS收入增长12%,是自2015年以来最慢的增速,但仍为亚马逊贡献了70%的营业利润。面对新竞争对手的挑战,亚马逊电商业务已显现复苏迹象。 亚马逊称7月份的Prime Day活动为有史以来最大规模,Prime会员共购买超过3.75亿件商品。首席执行官表示,通胀有所缓解,但消费者仍在降低消费。展望未来,亚马逊预计三季度销售额同比增长9%~13%。 投资者会议上,亚马逊表示在提升寄递时效和服务质量方面取得了里程碑式的成果。截至目前,通过当天达和次日达服务已经交付了18亿件商品,比2019年同期增长4倍。 过去4年,亚马逊投入资源缩短运输时间。现已基本恢复正常速度,实现“有史以来最快的Prime送货速度”。可享受免费Prime送货的商品数量已超3亿件。亚马逊不再采用全国性的配送网络,而是将全国划分为8个独立区域,由分拨设施完成订单。约76%的美国客户订单由这8个区域完成。 交货速度提升吸引更多购物者从线下转向亚马逊。但也增加了前期成本,一些投资者担心会吞噬利润。履单和运输成本的增长速度超过上季度。但亚马逊并未停止布局,计划未来2年内将当日达站点数量增加1倍。', 'question': '你是一个经济分析师，请对如下财报做分析给出你的洞察', '<ans>': '亚马逊二季度财报分析'}]
[{'input': '问题：我应该穿什么衣服出门？回答：亲爱的,外面好冷哦,记得穿厚厚的外套,还有奶奶给你织的那条温暖的围巾。手套、帽子也一定要戴上,把小手小脖子都裹得严严实实的,免得着了凉。去的时候一定要注意安全

/tmp/ipykernel_7504/2918657064.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r12[['input_tok', 'question_tok', 'ans_tok']] = r12.applymap(lambda x: len(tokenizer.tokenize(x)))


In [13]:
r12=pd.DataFrame.from_records(res12)
r12.to_csv("list12_result.csv", index=False)

In [15]:
import pandas as pd
l13=pd.read_csv("list13.csv", header=None, sep="\t")
l13.columns=["document"]
l13[['question', 'input']] = l13['document'].str.split(pat="#",n=1,expand=True)
res13 = []
for _ in range(0,3):
    for _, d in l13.iterrows():
        prompt = {"input":d["input"].replace("\r\n",""), "question":d["question"].replace("\r\n",""), "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=200, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res13.extend(inference_results)
        print(inference_results)

r13=pd.DataFrame.from_records(res13)
r13=r13.fillna("")
r13[['input_tok', 'question_tok', 'ans_tok']] = r13.applymap(lambda x: len(tokenizer.tokenize(x)))
r13.to_csv("list13_result.csv", index=False)

[{'input': '你好，人工智能助手，我想了解一下日本最近的经济发展情况。你知道现在去日本的旅游签证政策是什么？日本福岛的核废水排海会对周围的生态造成哪些影响？核辐射的级别分别都有什么？', 'question': '请对以下的内容做意图识别，输出只需要包括一种意图类别。可选的意图类别一共6个，下面是每个类别名字和类别对应的解释和含义。询问信息:用户想要获取或者检索某个特定的与产品或者服务相关的主题或问题的事实性信息和答案。寻求建议：用户遇到困难或者不知道该如何做决定的时候想要系统或者服务提供方给予某个与产品或服务相关的问题的具有建设性的建议或意见。阐述使用体验：用户向系统或服务提供方阐述其对产品或服务原生真实的使用体验。提出建议：用户根据自己已有的专业知识和理解，给系统或者服务提供方提出合理的产品或服务的优化方式和思路。闲聊：用户只是漫无目的讲述一些与产品和服务都无关的事情。其他：不属于前面5个类别中的任何一个。你只需要输出一种你认为最合适的分类。绝对不要输出对分类的解释，也不要输出任何语义分割的符号。', '<ans>': '询问信息'}]
[{'input': '我认为现在的网络购物真的太方便了，大大节省了我们的时间。无论是日常生活用品还是奢侈品，只需轻轻一点，商品就能送到家门口。这种便利性让我们不再需要亲自去商店排队等待结账，也省去了交通时间和麻烦。此外，网络购物还提供了更多的选择和比较的机会，我们可以在不同的网站上找到最合适的价格和品质。然而，对于明星代言的产品，我持保留意见。虽然明星代言可以增加产品的知名度和吸引力，但有些产品质量并不一定能保证。有时候，明星代言只是为了商业利益而存在，而不是真正为了产品的品质负责。因此，在购买明星代言的产品时，我们需要保持理性思考，仔细研究产品的评价和口碑。', 'question': '请对以下的内容做意图识别，输出只需要包括一种意图类别。可选的意图类别一共6个，下面是每个类别名字和类别对应的解释和含义:询问信息：用户想要获取或者检索某个特定的与产品或者服务相关的主题或问题的事实性信息和答案。寻求建议：用户遇到困难或者不知道该如何做决定的时候, 想要系统或者服务提供方给予某个与产品或服务相关的问题的具有建设性的建议或意见。阐述使用体验：用户向系统或服务提供方阐述其对产品或服务原生真实的使用体验。提出建议：用户根据

/tmp/ipykernel_7504/1350968851.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r13[['input_tok', 'question_tok', 'ans_tok']] = r13.applymap(lambda x: len(tokenizer.tokenize(x)))


In [18]:
# r13=pd.DataFrame.from_records(res13)
r13.to_csv("list13_result.csv", index=False)

In [17]:
l14=pd.read_csv("list14.csv")
l14
res14 = []
for _ in range(0,3):
    for _, d in l14.iterrows():
        prompt = {"input":d["instruction"].replace("\r\n",""), "question":d["question"].replace("\r\n",""), "<ans>":""}
        try:
            inference_results = llm.generate([prompt], max_length=200, repetition_penalty=1.1, beam_size=1)
        except Exception as e: 
            print(e)
            inference_results = [
                {"<ans>": "OOM"},
            ]
        res14.extend(inference_results)
        print(inference_results)

r14=pd.DataFrame.from_records(res14)
r14=r14.fillna("")
r14[['input_tok', 'question_tok', 'ans_tok']] = r14.applymap(lambda x: len(tokenizer.tokenize(x)))
r14.to_csv("list14_result.csv", index=False)

[{'input': '请对以下内容进行情感分析，请对于内容中表达的情感进行分类。\n分类的标签来自以下六种情感：喜悦，悲伤，愤怒，恐惧，厌恶，中性。这六种情感的定义如下：\n\n喜悦： 一种积极的情感，通常与快乐、满足和愉快相关。\n悲伤： 一种负面情感，通常与失落、悲痛和沮丧相关。\n愤怒： 一种情感，通常与愤怒、恼火和愤慨相关。\n恐惧： 一种负面情感，通常与焦虑、害怕和不安相关。\n厌恶： 一种情感，通常与厌恶、反感和恶心相关。\n中性：只描述事实，没有任何上面五种情感的直接表达。\n\n你只需要输出一种你认为最合适的分类。绝对不要输出对分类的解释，也不要输出任何的语义分割符号！', 'question': '\n"哇，我终于买到了我梦寐以求的 LV 挎包！我简直太开心了！这个挎包一直在我心里，每次看到它，\n我都觉得它是我生活中的一个小梦想。现在，终于拥有它了，我感觉就像实现了一个小愿望。\n它的设计、质感和细节都太棒了，我已经迫不及待要搭配它的时尚装扮，炫耀给朋友们看了。\n这真的是我最喜欢的礼物，我觉得自己特别幸运和满足！"\n', '<ans>': "'喜悦\n愤怒'"}]
[{'input': '请对以下内容进行情感分析，请对于内容中表达的情感进行分类。\n分类的标签来自以下六种情感：喜悦，悲伤，愤怒，恐惧，厌恶，中性。这六种情感的定义如下：\n\n喜悦： 一种积极的情感，通常与快乐、满足和愉快相关。\n悲伤： 一种负面情感，通常与失落、悲痛和沮丧相关。\n愤怒： 一种情感，通常与愤怒、恼火和愤慨相关。\n恐惧： 一种负面情感，通常与焦虑、害怕和不安相关。\n厌恶： 一种情感，通常与厌恶、反感和恶心相关。\n中性：只描述事实，没有任何上面五种情感的直接表达。\n\n你只需要输出一种你认为最合适的分类。绝对不要输出对分类的解释，也不要输出任何的语义分割符号！', 'question': '\n刚刚看完《断背山》，我感到非常难受。这部电影真是令人心碎，故事中的角色经历了如此多的磨难和困难，\n尤其是两位主角的命运，让我感到深深地同情。电影中的情感和剧情都令人难以承受，\n但同时也让我反思生活中的各种挑战和爱的重要性。这是一部深刻而感人的电影。\n', '<ans>': "'恐惧'"}]
[{'input': '请对以下内容进行情感分析，请对于内容中表达

/tmp/ipykernel_7504/1394449059.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  r14[['input_tok', 'question_tok', 'ans_tok']] = r14.applymap(lambda x: len(tokenizer.tokenize(x)))


In [19]:
r14.to_csv("list14_result.csv", index=False)